In [1]:
import pandas as pd 
import numpy as np
import requests 
import os

In [2]:
# Function to import data sets (with url)
def get_data(url):
    csv_url = url
    req = requests.get(csv_url)
    url_content = req.content
    csv_file = open('downloaded.csv', 'wb')
    csv_file.write(url_content)
    csv_file.close()
    df = pd.read_csv('downloaded.csv')
    os.remove("downloaded.csv")
    return df

In [3]:
# Export dataset Cases|Deads|Rate weekly
df_cases_deaths_rate = get_data("https://opendata.ecdc.europa.eu/covid19/nationalcasedeath/csv")
#df_mobility = get_data("https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv")
#df_country_code = get_data("https://datahub.io/JohnSnowLabs/list-of-iso-3166-1-alpha-2-english-country-names-and-code-elements/r/list-of-iso-3166-1-alpha-2-english-country-names-and-code-elements-csv.csv")
#df_vacine = get_data("https://opendata.ecdc.europa.eu/covid19/vaccine_tracker/csv/data.csv")
#df_testing = get_data("https://opendata.ecdc.europa.eu/covid19/testing/csv")
#df_response_measures = get_data("https://www.ecdc.europa.eu/sites/default/files/documents/response_graphs_data_2021-04-15.csv")
#df_age = get_data("https://opendata.ecdc.europa.eu/covid19/agecasesnational/csv")

#Delete Columns 
df_cases_deaths_rate = df_cases_deaths_rate.drop(['source'], axis=1)
#Filter Dataset Cases
df_cases = df_cases_deaths_rate[df_cases_deaths_rate['indicator']=='cases']
#Filter Dataset Deaths
df_deaths = df_cases_deaths_rate[df_cases_deaths_rate['indicator']=='deaths']

#rate_14_day -> 14-day notification rate of reported COVID-19 cases per 100000 population 
#OR 14-day notification rate of reported deaths per 1 000 000 population


In [4]:
#Prepare Dataset with df_cases
df_model = df_cases
#Separet year and week
df_model['year'] = df_model['year_week'].str[:4]
df_model['week'] = df_model['year_week'].str[5:8]
#Create Dependent Variable
calc_week = df_model.drop(['continent','population',
                           'indicator','rate_14_day','cumulative_count'], axis=1)

calc_week = calc_week.rename(columns={'weekly_count': 'next_week_count'})
calc_week['week'] = calc_week['week'].apply(pd.to_numeric)
calc_week['year'] = calc_week['year'].apply(pd.to_numeric)
calc_week['week'] = calc_week['week']-1
calc_week.loc[calc_week['week'] == 0, 'year'] = calc_week['year']-1
calc_week.loc[calc_week['week'] == 0, 'week'] = 53

#Create id to join dataset
calc_week['id'] = calc_week['country'] +  calc_week['year'].astype(str) + calc_week['week'].astype(str)
calc_week = calc_week.drop(['country','country_code',
                           'year','week'], axis=1)
df_model['week'] = df_model['week'].apply(pd.to_numeric)
df_model['year'] = df_model['year'].apply(pd.to_numeric)
df_model['id'] = df_model['country'] +  df_model['year'].astype(str) + df_model['week'].astype(str)

#Left join and delete duplicate rows
df_model = pd.merge(df_model,calc_week, left_on='id', right_on = 'id', how='left', indicator=True)
df_model = df_model.drop_duplicates(subset="id")

#Rename Columns 
df_model = df_model.rename(columns={'weekly_count': 'last_week_cases', 'year_week_x':'year_last_week',
                                    'rate_14_day': 'last_week_rate_14_day','year_week_y':'week_predicton',
                                    'next_week_count':'week_cases'
                                    , 'cumulative_count':'cumulative_cases_until_last_week'})
#Create dependent
df_model['variation_cases']=df_model['week_cases']-df_model['last_week_cases']
df_model['Increased_cases']=np.where(df_model['week_cases']>df_model['last_week_cases'], 1, 0)
#Drop rows with left_only
df_model = df_model[df_model['_merge']=='both']
#Remove Columns
df_model = df_model.drop(['indicator',
                           'year','week','id','_merge','last_week_rate_14_day'], axis=1)

print(df_model)

/home/luis/anaconda3/envs/covid_vizualization/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/luis/anaconda3/envs/covid_vizualization/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/luis/anaconda3/envs/covid_vizualization/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

           country country_code continent  population  last_week_cases  \
0      Afghanistan          AFG      Asia    38928341              0.0   
1      Afghanistan          AFG      Asia    38928341              0.0   
2      Afghanistan          AFG      Asia    38928341              0.0   
3      Afghanistan          AFG      Asia    38928341              0.0   
4      Afghanistan          AFG      Asia    38928341              0.0   
...            ...          ...       ...         ...              ...   
14940     Zimbabwe          ZWE    Africa    14862927            138.0   
14941     Zimbabwe          ZWE    Africa    14862927            141.0   
14942     Zimbabwe          ZWE    Africa    14862927            122.0   
14943     Zimbabwe          ZWE    Africa    14862927            262.0   
14944     Zimbabwe          ZWE    Africa    14862927            245.0   

      year_last_week  cumulative_cases_until_last_week  week_cases  \
0            2020-01                     

/home/luis/anaconda3/envs/covid_vizualization/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/luis/anaconda3/envs/covid_vizualization/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
#Join deaths with df_model
#LAST WEEK
df_join = df_deaths
df_join['ID'] = df_join['country']+df_join['year_week']
df_join = df_join.drop(['year_week','country','indicator','country_code','continent'
                        ,'population','rate_14_day'], axis=1)
df_join = df_join.rename(columns={'weekly_count': 'last_week_deaths'
                                  ,'cumulative_count':'cumulative_deaths_until_last_week'})
df_model['ID'] = df_model['country']+df_model['year_last_week']
df_model = pd.merge(df_model,df_join, left_on='ID', right_on = 'ID', how='left', indicator=False)
df_model = df_model.drop(['ID'], axis=1)

#WEEK PREDICTON
df_join = df_deaths
df_join['ID'] = df_join['country']+df_join['year_week']
df_join = df_join.drop(['cumulative_count','year_week','country','indicator','country_code'
                        ,'continent','population','rate_14_day'], axis=1)
df_join = df_join.rename(columns={'weekly_count': 'week_deaths'})
df_model['ID'] = df_model['country']+df_model['week_predicton']
df_model = pd.merge(df_model,df_join, left_on='ID', right_on = 'ID', how='left', indicator=False)
df_model = df_model.drop(['ID'], axis=1)
df_model['variation_deaths']=df_model['week_deaths']-df_model['last_week_deaths']


/home/luis/anaconda3/envs/covid_vizualization/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/luis/anaconda3/envs/covid_vizualization/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [6]:
#Mobility Country

#Remove data of sub_region
df_mobility_group = df_mobility[df_mobility[['sub_region_1']].isnull().any(1)]
#Create ID week-year
df_mobility_group['date'] = pd.to_datetime(df_mobility_group['date'])
df_mobility_group['week'] =  df_mobility_group['date'].dt.week
df_mobility_group['year'] =  (df_mobility_group['date'].dt.year).apply(str)
df_mobility_group['week'] = np.where(df_mobility_group['week'].apply(str).str.len() < 2
                                      , "0" + (df_mobility_group['week']).apply(str)
                                      ,(df_mobility_group['week']).apply(str))
df_mobility_group['ID'] = df_mobility_group['country_region']+df_mobility_group['year']+"-"+df_mobility_group['week']
#Group by
df_mobility_group = df_mobility_group.groupby('ID',as_index=False)['retail_and_recreation_percent_change_from_baseline'
                                                                   ,'grocery_and_pharmacy_percent_change_from_baseline'
                                                                   ,'parks_percent_change_from_baseline'
                                                                   ,'transit_stations_percent_change_from_baseline'
                                                                   ,'workplaces_percent_change_from_baseline'
                                                                   ,'residential_percent_change_from_baseline'].mean()
#Join df model
df_model['ID'] = df_model['country']+df_model['year_last_week']
df_model = pd.merge(df_model,df_mobility_group, left_on='ID', right_on = 'ID', how='left', indicator=False)
df_model = df_model.drop(['ID'], axis=1)
#Remove countries without mobility information
dfcountry = df_model.groupby('country',as_index=False).count()
dfcountry['nulos'] = dfcountry['retail_and_recreation_percent_change_from_baseline']+dfcountry['grocery_and_pharmacy_percent_change_from_baseline']+dfcountry['parks_percent_change_from_baseline']+dfcountry['workplaces_percent_change_from_baseline']+dfcountry['residential_percent_change_from_baseline']
dfcountry = dfcountry.sort_values(by = ['nulos'])
dfcountry = dfcountry[dfcountry['nulos'] > 0]
dfcountry = dfcountry['country']
dfcountry.columns = ['country']
df_model = pd.merge(df_model,dfcountry, left_on='country', right_on = 'country', how='inner', indicator=False)

NameError: name 'df_mobility' is not defined

In [ ]:
#Vacination 
#Insert Names of Countries
df_vacine = pd.merge(df_vacine,df_country_code, left_on='ReportingCountry', right_on = 'ISO3166_Alpha2_Code', how='left', indicator=False)
#Create ID
df_vacine['ID'] = df_vacine['Country_Name']+df_vacine['YearWeekISO'].str[:4]+"-"+df_vacine['YearWeekISO'].str[6:8]
#Group by
df_vacine = df_vacine.groupby('ID',as_index=False)['FirstDose','FirstDoseRefused','SecondDose','UnknownDose'].sum()
#Join
df_model['ID'] = df_model['country']+df_model['year_last_week']
df_model = pd.merge(df_model,df_vacine, left_on='ID', right_on = 'ID', how='left', indicator=False)
df_model = df_model.drop(['ID'], axis=1)

In [ ]:
#Nulos 
df_model['retail_and_recreation_percent_change_from_baseline'] = df_model['retail_and_recreation_percent_change_from_baseline'].fillna(0)
df_model['grocery_and_pharmacy_percent_change_from_baseline'] = df_model['grocery_and_pharmacy_percent_change_from_baseline'].fillna(0)
df_model['parks_percent_change_from_baseline'] = df_model['parks_percent_change_from_baseline'].fillna(0)
df_model['transit_stations_percent_change_from_baseline'] = df_model['transit_stations_percent_change_from_baseline'].fillna(0)
df_model['workplaces_percent_change_from_baseline'] = df_model['workplaces_percent_change_from_baseline'].fillna(0)
df_model['residential_percent_change_from_baseline'] = df_model['residential_percent_change_from_baseline'].fillna(0)
df_model['FirstDose'] = df_model['FirstDose'].fillna(0)
df_model['FirstDoseRefused'] = df_model['FirstDoseRefused'].fillna(0)
df_model['SecondDose'] = df_model['SecondDose'].fillna(0)
df_model['UnknownDose'] = df_model['UnknownDose'].fillna(0)

In [ ]:
#Index and organize dataset
df_model_dl = df_model[["week_predicton","country","continent","population","last_week_cases",
                        "cumulative_cases_until_last_week","last_week_deaths","cumulative_deaths_until_last_week",
                        "retail_and_recreation_percent_change_from_baseline",
                        "grocery_and_pharmacy_percent_change_from_baseline","parks_percent_change_from_baseline",
                        "transit_stations_percent_change_from_baseline","workplaces_percent_change_from_baseline",
                        "residential_percent_change_from_baseline","FirstDose","FirstDoseRefused","SecondDose",
                        "UnknownDose","Increased_cases"]]
df_model_dl = df_model_dl.set_index('week_predicton')
print(df_model_dl)

In [ ]:

#Situação Atual 
#-> Ranking Países (Mortes e Casos Ultimos 15 dias)
#-> %Pop Vacinada 
#-> Média de %Pop vacinada por dia nos ultimos 15 dias) -> Quando atinge 70% ? 
#-> Gráfico Mapa com Análise de Mobilidade por País 
#-> Internados nas 2 ultimas semanas (lotação????)

#Análise Histórico (COmparação 2 Países)
#Filtros ??2?? Países (Data)
#-> Mortes 
#-> Casos 
#-> Vacinação 
#-> Média Mobilidade por tipo

#Rede Neuronal 
#-> Previsão da Próxima semana (aumento de casos) (Rede Neuronal)
#-> Análise de resultados de Redes Neunonais 
#-> Explicabilidade das Redes Neuronais


# -> Momento Atual 
# -> Já ta feito: Recolha de Datasets ; Análise de Artigos; Escolha do Output(Indicadores)
# -> Em desenvolvimento : Análise das fontes de dados; Tratamento de dados; desenvolvimento das vizualizações dos dados
# -> Futuro : Desenvolvimento Metedologico da Rede Neuronal; Desenvolvimento da Interface Web; 

In [ ]:
#Statistics Dataset Model
print("information metadata: \n")
print(df_model_dl.info(verbose=True))
print("\n descripton dataset: \n")
print(df_model_dl.describe())
print("\n country in dataset: \n")
x = df_model_dl.groupby(['country']).agg(['count']).index
for i in x:
    print(i)


In [ ]:
#export to csv 
df_model_dl.to_csv(r'dataset.csv', index = True)
df_model.to_csv(r'full_dataset.csv', index= False)